In [1]:
import pandas as pd
import requests
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import regex as re
import numpy as np
import modin.pandas as mp

path = '/home/emmanuel/Desktop/descarga_db/raw_data_project_m1.db'
engine = create_engine(f'sqlite:///{path}')
raw_df = pd.read_sql_query("SELECT * FROM sqlite_master WHERE type='table'", engine)


In [2]:
#query to get the df information

df_personal = pd.read_sql('personal_info',engine)
df_country = pd.read_sql('country_info',engine)
df_career = pd.read_sql('career_info',engine)
df_poll = pd.read_sql('poll_info',engine)


In [6]:
#print(df_personal.shape)
#print(df_country.shape)
#print(df_career.shape)
#print(df_poll.shape)

In [3]:
engine.dispose()

In [5]:
#df_personal.head(1)

In [4]:
def gender_modifier(x):
    if x == 'FeMale':
        return 'Female'
    elif x == 'Fem':
        return 'Female'
    elif x == 'female':
        return 'Female'
    elif x == 'male':
        return 'Male'
    elif x == 'Male':
        return 'Male'

In [5]:
df_personal['dem_has_children'].unique()

array(['NO', 'yES', 'nO', 'YES'], dtype=object)

In [6]:
def children_modifier(x):
    if x == 'NO':
        return 'no'
    elif x == 'nO':
        return 'no'
    elif x == 'No':
        return 'no'
    elif x == 'YES':
        return 'yes'
    elif x == 'yES':
        return 'yes'

In [7]:
#appliying my definition to clean gender and dem_has_children columns
df_personal['gender_modified'] = df_personal['gender'].apply(gender_modifier)
df_personal['dem_has_children_mo'] = df_personal['dem_has_children'].apply(children_modifier)

#from df raw, dropping the old and unclean 'gender' and dem_has_children columns
df_personal_clean = df_personal.drop(columns=['gender','dem_has_children'])


#rename column
df_personal_clean = df_personal_clean.rename(columns={'gender_modified':'gender','dem_has_children_mo':'dem_has_children'})

In [8]:
df_personal_clean.isnull().sum()#no nulls
df_personal_clean.head()
df_personal_clean['age_group'].unique()

array(['40_65', '26_39', 'juvenile', '14_25'], dtype=object)

In [9]:
df_personal_clean.to_csv('../raw_data/df_personal.csv',index=False)

In [22]:
#country info dataframe

In [10]:
df_country.isnull().sum()# no nulls
df_country.to_csv('../raw_data/df_country.csv',index=False)#sent them to raw data folder

In [11]:
#career info dataframe
df_career['dem_education_level'].unique()

array(['no', 'high', None, 'medium', 'low'], dtype=object)

In [12]:
df_career

,uuid,dem_education_level,dem_full_time_job,normalized_job_code
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f
...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,high,yes,847165cfda6b1dc82ae22b967da8af2f
9645,39f989f0-db52-0133-8482-0a81e8b09a82,high,yes,a4d5b8b38f9513825d0d94a981ebe962
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,low,no,None
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,low,yes,775190277a849cba701b306a7b374c0a


In [13]:
df_career['normalized_job_code'].fillna('None',inplace=True)
df_career['dem_education_level'].fillna('None',inplace=True)

In [14]:
df_career.to_csv('../raw_data/df_career.csv',index=False)

In [15]:
#poll info dataframe

In [16]:
df_poll.isnull().sum()

uuid                                                   0
question_bbi_2016wave4_basicincome_awareness           0
question_bbi_2016wave4_basicincome_vote                0
question_bbi_2016wave4_basicincome_effect              0
question_bbi_2016wave4_basicincome_argumentsfor        0
question_bbi_2016wave4_basicincome_argumentsagainst    0
dtype: int64

In [17]:
df_poll.to_csv('../raw_data/df_poll.csv',index=False)

In [19]:
jobs_list = df_career['normalized_job_code'].unique()
jobs_raw_uni = pd.DataFrame(jobs_list)
jobs_raw_uniques = jobs_raw_uni.rename(columns={0:'normalized_job_code'})

In [20]:
#df_career
jobs_raw_uniques

,normalized_job_code
0,None
1,861a9b9151e11362eb3c77ca914172d0
2,049a3f3a2b5f85cb2971ba77ad66e10c
3,f4b2fb1aa40f661488e2782b6d57ad2f
4,27af8700f5577cec835acee2cb90a2ff
...,...
152,b0fa6ede410f50b82ab74f5a705fe699
153,559a21f836c93876f31b60e6d10656a7
154,c1fb1a01b78373ac2153c66fa08d16dc
155,05bb9a333a66d6eb151e253623efe1c0


In [21]:

def api_caller(x):
    req=[]
    r = requests.get('http://api.dataatwork.org/v1/jobs/' + str(x))
    req = r.json()
    return req['normalized_job_title']


In [22]:
#df_career['new_job_code'] = df_career['normalized_job_code'].apply(api_caller)
#api_caller('ac47656fd51c2cd8037057262c910dc4')

jobs_raw_uniques['job_name'] = jobs_raw_uniques.iloc[1:,0].apply(api_caller)

jobs_raw_uniques 

,normalized_job_code,job_name
0,None,NaN
1,861a9b9151e11362eb3c77ca914172d0,automatic data processing planner
2,049a3f3a2b5f85cb2971ba77ad66e10c,data coordinator
3,f4b2fb1aa40f661488e2782b6d57ad2f,database developer
4,27af8700f5577cec835acee2cb90a2ff,data entry specialist
...,...,...
152,b0fa6ede410f50b82ab74f5a705fe699,analytical data miner
153,559a21f836c93876f31b60e6d10656a7,data analysis assistant
154,c1fb1a01b78373ac2153c66fa08d16dc,data examination clerk
155,05bb9a333a66d6eb151e253623efe1c0,data entry clerk


In [23]:
job_name_df = pd.DataFrame(jobs_raw_uniques)


job_name_df.to_csv('../raw_data/jobs_dataframe.csv')

In [24]:
#country codes

country_list = df_country['country_code'].unique().tolist()
country_list

['AT',
 'BE',
 'BG',
 'CY',
 'CZ',
 'DE',
 'DK',
 'EE',
 'ES',
 'FI',
 'FR',
 'GB',
 'GR',
 'HR',
 'HU',
 'IE',
 'IT',
 'LT',
 'LU',
 'LV',
 'MT',
 'NL',
 'PL',
 'PT',
 'RO',
 'SE',
 'SI',
 'SK']

In [ ]:
#pd.DataFrame(country_dict.items(),columns=['country_name','country_code']).to_csv('../raw_data/country_codes.csv')

In [25]:
url = 'https://www.iban.com/country-codes' 
raw_codes = pd.read_html(url,header=0)[0]
raw_codes

#only keeping columns that are needed
country_codes = raw_codes.drop(columns=['Alpha-3 code','Numeric'])
country_code = country_codes.rename(columns={'Country':'country','Alpha-2 code':'country_code'})
country_code

country_code.to_csv('../raw_data/country_codes.csv',index=False)


In [141]:
#country codes through webscrapping

""""url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes' 
html = requests.get(url).content
soup = BeautifulSoup(html, 'lxml')
#parsing and getting the table
countries = soup.find_all('td')

country_raw_lst = []
for item in countries:
    if str(item) != '':
        country_raw_lst.append(str(item))

country_raw_lst[0::1]"""






'"url = \'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes\' \nhtml = requests.get(url).content\nsoup = BeautifulSoup(html, \'lxml\')\n#parsing and getting the table\ncountries = soup.find_all(\'td\')\n\ncountry_raw_lst = []\nfor item in countries:\n    if str(item) != \'\':\n        country_raw_lst.append(str(item))\n\ncountry_raw_lst[0::1]'

In [142]:
""""#turning into a df
country_table = pd.DataFrame(columns=['country_code', 'country'])
country_table

row_maker = 0

for row in table.find_all('tr'):
    column_maker = []
    columns = row.find_all('td')
    for column in columns:
        country_table.iat[row_maker,column_maker]= column.get_text()
        column_maker +=1
    
new_table    

#only keeping columns that are needed
#country_codes = raw_codes.drop(columns=['Alpha-3 code','Numeric'])
#country_code = country_codes.rename(columns={'Country':'country','Alpha-2 code':'country_code'})"""

#saving all the country codes



'"#turning into a df\ncountry_table = pd.DataFrame(columns=[\'country_code\', \'country\'])\ncountry_table\n\nrow_maker = 0\n\nfor row in table.find_all(\'tr\'):\n    column_maker = []\n    columns = row.find_all(\'td\')\n    for column in columns:\n        country_table.iat[row_maker,column_maker]= column.get_text()\n        column_maker +=1\n    \nnew_table    \n\n#only keeping columns that are needed\n#country_codes = raw_codes.drop(columns=[\'Alpha-3 code\',\'Numeric\'])\n#country_code = country_codes.rename(columns={\'Country\':\'country\',\'Alpha-2 code\':\'country_code\'})'

In [26]:
#Merging countries and country codes
countries_merged = pd.merge(country_code,df_country,how='right',on='country_code')

countries_merged .to_csv('../raw_data/country_info_merged.csv',index=False)


In [27]:
full_personal = pd.merge(df_personal,countries_merged,how='left',on='uuid')
full_personal

,uuid,age,gender,dem_has_children,age_group,gender_modified,dem_has_children_mo,country,country_code,rural
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,male,NO,40_65,Male,no,Austria,AT,countryside
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65,Male,yes,Austria,AT,urban
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,male,nO,26_39,Male,no,Austria,AT,city
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,Male,YES,40_65,Male,yes,Austria,AT,Country
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,Fem,yES,40_65,Female,yes,Austria,AT,city
...,...,...,...,...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,37 years old,FeMale,nO,26_39,Female,no,Slovakia,SK,urban
9645,39f989f0-db52-0133-8482-0a81e8b09a82,53 years old,Male,yES,40_65,Male,yes,Slovakia,SK,urban
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,1992,male,NO,juvenile,Male,no,Slovakia,SK,Non-Rural
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,47 years old,male,yES,40_65,Male,yes,Slovakia,SK,city


In [28]:
merged_interview = pd.merge(full_personal,df_poll,how='left',on='uuid')

In [29]:
merged_interview


,uuid,age,gender,dem_has_children,age_group,gender_modified,dem_has_children_mo,country,country_code,rural,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,male,NO,40_65,Male,no,Austria,AT,countryside,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65,Male,yes,Austria,AT,urban,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,male,nO,26_39,Male,no,Austria,AT,city,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,Male,YES,40_65,Male,yes,Austria,AT,Country,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,Fem,yES,40_65,Female,yes,Austria,AT,city,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,37 years old,FeMale,nO,26_39,Female,no,Slovakia,SK,urban,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance
9645,39f989f0-db52-0133-8482-0a81e8b09a82,53 years old,Male,yES,40_65,Male,yes,Slovakia,SK,urban,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,1992,male,NO,juvenile,Male,no,Slovakia,SK,Non-Rural,I have heard just a little about it,I would not vote,‰Û_ spend more time with my family,It reduces anxiety about financing basic needs,None of the above
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,47 years old,male,yES,40_65,Male,yes,Slovakia,SK,city,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...


In [30]:
full_poll = pd.merge(merged_interview,df_career,how='left',on='uuid')

In [31]:
full_poll

,uuid,age,gender,dem_has_children,age_group,gender_modified,dem_has_children_mo,country,country_code,rural,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,dem_education_level,dem_full_time_job,normalized_job_code
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,male,NO,40_65,Male,no,Austria,AT,countryside,I know something about it,I would not vote,None of the above,None of the above,None of the above,no,no,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65,Male,yes,Austria,AT,urban,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,high,yes,861a9b9151e11362eb3c77ca914172d0
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,male,nO,26_39,Male,no,Austria,AT,city,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...,None,no,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,Male,YES,40_65,Male,yes,Austria,AT,Country,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,Fem,yES,40_65,Female,yes,Austria,AT,city,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,37 years old,FeMale,nO,26_39,Female,no,Slovakia,SK,urban,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance,high,yes,847165cfda6b1dc82ae22b967da8af2f
9645,39f989f0-db52-0133-8482-0a81e8b09a82,53 years old,Male,yES,40_65,Male,yes,Slovakia,SK,urban,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...,high,yes,a4d5b8b38f9513825d0d94a981ebe962
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,1992,male,NO,juvenile,Male,no,Slovakia,SK,Non-Rural,I have heard just a little about it,I would not vote,‰Û_ spend more time with my family,It reduces anxiety about financing basic needs,None of the above,low,no,None
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,47 years old,male,yES,40_65,Male,yes,Slovakia,SK,city,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...,low,yes,775190277a849cba701b306a7b374c0a


final_df = pd.merge(full_poll,job_name_df,how='left',on='normalized_job_code')
#final_df.to_csv('../raw_data/full_dataframe.csv',index=False)
df_modified = final_df.drop(columns='gender')
#full_set = df_modified.rename(columns= {'gender_modifed':'gender'})


In [61]:
final_df = pd.merge(full_poll,job_name_df,how='left',on='normalized_job_code') 
#final_df.to_csv('../raw_data/full_dataframe.csv',index=False) 
df_modified = final_df.drop(columns=['gender','dem_has_children']) 
df_modified

,uuid,age,age_group,gender_modified,dem_has_children_mo,country,country_code,rural,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,dem_education_level,dem_full_time_job,normalized_job_code,job_name
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,40_65,Male,no,Austria,AT,countryside,I know something about it,I would not vote,None of the above,None of the above,None of the above,no,no,None,NaN
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,40_65,Male,yes,Austria,AT,urban,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,high,yes,861a9b9151e11362eb3c77ca914172d0,automatic data processing planner
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,26_39,Male,no,Austria,AT,city,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...,None,no,None,NaN
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,40_65,Male,yes,Austria,AT,Country,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,data coordinator
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,40_65,Female,yes,Austria,AT,city,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,database developer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,37 years old,26_39,Female,no,Slovakia,SK,urban,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance,high,yes,847165cfda6b1dc82ae22b967da8af2f,data warehouse developer
9645,39f989f0-db52-0133-8482-0a81e8b09a82,53 years old,40_65,Male,yes,Slovakia,SK,urban,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...,high,yes,a4d5b8b38f9513825d0d94a981ebe962,database manager
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,1992,juvenile,Male,no,Slovakia,SK,Non-Rural,I have heard just a little about it,I would not vote,‰Û_ spend more time with my family,It reduces anxiety about financing basic needs,None of the above,low,no,None,NaN
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,47 years old,40_65,Male,yes,Slovakia,SK,city,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...,low,yes,775190277a849cba701b306a7b374c0a,data officer


In [62]:
df_modified

,uuid,age,age_group,gender_modified,dem_has_children_mo,country,country_code,rural,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,dem_education_level,dem_full_time_job,normalized_job_code,job_name
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,40_65,Male,no,Austria,AT,countryside,I know something about it,I would not vote,None of the above,None of the above,None of the above,no,no,None,NaN
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,40_65,Male,yes,Austria,AT,urban,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,high,yes,861a9b9151e11362eb3c77ca914172d0,automatic data processing planner
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,26_39,Male,no,Austria,AT,city,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...,None,no,None,NaN
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,40_65,Male,yes,Austria,AT,Country,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,data coordinator
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,40_65,Female,yes,Austria,AT,city,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,database developer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,37 years old,26_39,Female,no,Slovakia,SK,urban,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance,high,yes,847165cfda6b1dc82ae22b967da8af2f,data warehouse developer
9645,39f989f0-db52-0133-8482-0a81e8b09a82,53 years old,40_65,Male,yes,Slovakia,SK,urban,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...,high,yes,a4d5b8b38f9513825d0d94a981ebe962,database manager
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,1992,juvenile,Male,no,Slovakia,SK,Non-Rural,I have heard just a little about it,I would not vote,‰Û_ spend more time with my family,It reduces anxiety about financing basic needs,None of the above,low,no,None,NaN
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,47 years old,40_65,Male,yes,Slovakia,SK,city,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...,low,yes,775190277a849cba701b306a7b374c0a,data officer


In [65]:
full_set = df_modified.rename(columns= {'gender_modified':'gender','dem_has_children_mo':'dem_has_children'})


In [68]:
full_set.to_csv('../raw_data/full_set.csv',index=False)

In [92]:
#making challenge number 1  


,Country,Job Title,Gender,Quantity
0,Austria,automatic data processing customer liaison adp...,Male,2
1,Austria,automatic data processing planner,Female,1
2,Austria,automatic data processing planner,Male,1
3,Austria,automatic data processing systems security spe...,Male,2
4,Austria,clinical data specialist,Male,1
...,...,...,...,...
2581,United Kingdom of Great Britain and Northern I...,scada operator supervisory control and data ac...,Female,1
2582,United Kingdom of Great Britain and Northern I...,scada technician supervisory control and data ...,Male,1
2583,United Kingdom of Great Britain and Northern I...,sql database administrator,Female,3
2584,United Kingdom of Great Britain and Northern I...,sql database administrator,Male,5
